# Importing Libraries and Data Loading

In [4]:
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split

In [29]:
ratings_ = pd.read_csv('dataset/ratings.csv')
movies_ = pd.read_csv('dataset/movies.csv')
ratings_ = pd.merge(movies_,ratings_).drop(['genres','timestamp'],axis=1)
print(ratings_.shape)
ratings_.head()

(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [30]:
ratings = ratings_.pivot_table(index=['userId'],columns=['title'],values='rating')
ratings.head()
print("Before: ",ratings.shape)

Before:  (610, 9719)


In [31]:
ratings

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


In [ ]:
first_row = ratings.iloc[609, :].items()
result = [(title, rating) for title, rating in first_row if rating != 0.0]
result

In [33]:
ratings = ratings.dropna(thresh=10, axis=1).fillna(0,axis=1)
print("After: ",ratings.shape)

After:  (610, 2269)


In [34]:
corrMatrix = ratings.corr(method='pearson')

In [49]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [50]:
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies = pd.DataFrame(columns = ratings.columns)
for movie,rating in romantic_lover:
    similar_movies.loc[len(similar_movies)] = get_similar(movie,rating)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(10)

title
(500) Days of Summer (2009)       2.584556
Alice in Wonderland (2010)        1.395229
Silver Linings Playbook (2012)    1.254800
Yes Man (2008)                    1.116264
Adventureland (2009)              1.112235
Marley & Me (2008)                1.108381
About Time (2013)                 1.102192
Crazy, Stupid, Love. (2011)       1.088757
50/50 (2011)                      1.086517
Help, The (2011)                  1.075963
dtype: float64

In [63]:
a = get_similar("Alice in Wonderland (2010)",5)
a.head(6).index.tolist()

['Alice in Wonderland (2010)',
 'Charlie and the Chocolate Factory (2005)',
 'National Treasure: Book of Secrets (2007)',
 'Madagascar (2005)',
 'Hobbit: An Unexpected Journey, The (2012)',
 'Up in the Air (2009)']

# Saving Data

In [64]:
import pickle
pickle.dump(corrMatrix,open('similarityMatrix.pkl','wb'))
pickle.dump(ratings,open('movieRatings.pkl','wb'))

In [36]:
# precision_list = []
# recall_list = []
# f1_list = []




# for k in range (10,150,10):
#     precisions, recalls, f1_scores = 0, 0, 0
#     for rows in range(len(ratings)):
#         # to get actual movies can be contain in actual top-k-rated movies
#         actual_ratings = [(title, rating) for title, rating in ratings.iloc[rows, 1:].items() if rating != 0.0]
#         actual_ratings_sorted = sorted(actual_ratings, key=lambda x: x[1], reverse=True)
#         top_k_min_rating = min([rating for _, rating in actual_ratings_sorted[:k]])
#         Actual_top_rated_movies = [movie for movie, rating in actual_ratings_sorted if rating >= top_k_min_rating]
    
#         # to predict to k movies
#         similar_movies = pd.DataFrame(columns = ratings.columns)
#         for movie,rating in actual_ratings_sorted:
#             similar_movies.loc[len(similar_movies)] = get_similar(movie,rating)
        
#         top_predicted_movies_Series = similar_movies.sum().sort_values(ascending=False).head(10)
#         top_predicted_movies = top_predicted_movies_Series.index.tolist()
    
#         true_positives = len(set(Actual_top_rated_movies) & set(top_predicted_movies))
#         false_positives = len(set(top_predicted_movies) - set(Actual_top_rated_movies))
#         false_negatives = len(set(Actual_top_rated_movies) - set(top_predicted_movies))
    
#         precision = 0 if true_positives == 0 else true_positives / (true_positives + false_positives)
#         recall = 0 if true_positives == 0 else true_positives / (true_positives + false_negatives)
#         f1_score = 0 if true_positives == 0 else 2 * precision * recall / (precision + recall)
        
#         precisions += precision
#         recalls += recall
#         f1_scores += f1_score
    
#     print ("K =", k, "Avg. Precision", precisions/len(ratings), "Avg. Recall", recalls/len(ratings), "Avg_f1", f1_scores/len(ratings))
#     precision_list.append(precisions/len(ratings))
#     recall_list.append(recalls/len(ratings))
#     f1_list.append(f1_scores/len(ratings))
        

In [37]:
# # when K becomes large, precision increases, recall decreases, overall f1 score decreases

# K = 10 Avg. Precision 0.5118032786885247 Avg. Recall 0.27829099245093675 Avg_f1 0.3418406257648635
# K = 20 Avg. Precision 0.6198360655737712 Avg. Recall 0.22303528457287738 Avg_f1 0.31956680277176674
# K = 30 Avg. Precision 0.6845901639344263 Avg. Recall 0.19694687202989572 Avg_f1 0.2971594419343499
# K = 40 Avg. Precision 0.7227868852459013 Avg. Recall 0.1865104962975877 Avg_f1 0.2858032992071915
# K = 50 Avg. Precision 0.7567213114754088 Avg. Recall 0.1807619938361781 Avg_f1 0.27913129746082105
# K = 60 Avg. Precision 0.7768852459016385 Avg. Recall 0.17632725578295894 Avg_f1 0.27304363177308033
# K = 70 Avg. Precision 0.7911475409836057 Avg. Recall 0.1736202865105 Avg_f1 0.269092315241546
# K = 80 Avg. Precision 0.8047540983606548 Avg. Recall 0.17141186607961123 Avg_f1 0.2657439718040974

In [38]:
# k = 10
# precisions, recalls, f1_scores = 0, 0, 0
# for rows in range(len(ratings)):
#         # to get actual movies can be contain in actual top-k-rated movies
#         actual_ratings = [(title, rating) for title, rating in ratings.iloc[rows, :].items() if rating != 0.0]
#         actual_ratings_sorted = sorted(actual_ratings, key=lambda x: x[1], reverse=True)
#         top_k_min_rating = min([rating for _, rating in actual_ratings_sorted[:k]])
#         Actual_top_rated_movies = [movie for movie, rating in actual_ratings_sorted if rating >= top_k_min_rating]
    
#         # to predict to k movies
#         similar_movies = pd.DataFrame(columns = ratings.columns)
#         for movie,rating in actual_ratings_sorted:
#             similar_movies.loc[len(similar_movies)] = get_similar(movie,rating)
        
#         top_predicted_movies_Series = similar_movies.sum().sort_values(ascending=False).head(10)
#         top_predicted_movies = top_predicted_movies_Series.index.tolist()
    
#         true_positives = len(set(Actual_top_rated_movies) & set(top_predicted_movies))
#         false_positives = len(set(top_predicted_movies) - set(Actual_top_rated_movies))
#         false_negatives = len(set(Actual_top_rated_movies) - set(top_predicted_movies))
    
#         precision = 0 if true_positives == 0 else true_positives / (true_positives + false_positives)
#         recall = 0 if true_positives == 0 else true_positives / (true_positives + false_negatives)
#         f1_score = 0 if true_positives == 0 else 2 * precision * recall / (precision + recall)
        
#         precisions += precision
#         recalls += recall
#         f1_scores += f1_score
    
# print ("K =", k, "Avg. Precision", precisions/len(ratings), "Avg. Recall", recalls/len(ratings), "Avg_f1", f1_scores/len(ratings))

In [40]:
# action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
# similar_movies = pd.DataFrame()
# for movie,rating in action_lover:
#     similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

# similar_movies.head(10)
# similar_movies.sum().sort_values(ascending=False).head(20)

In [41]:
# all_ratings = {}
# sum_ratings, sum_responses = 0, 0

# for i in range(len(ratings.columns)):
#     row = ratings.iloc[:, i].items()
#     curr_ratings = [ rating for title, rating in row if rating != 0.0]
#     sum_curr = sum(curr_ratings)
#     no_curr = len(curr_ratings)
#     sum_ratings += sum_curr
#     sum_responses += no_curr
#     all_ratings[ratings.columns[i]] = (sum_curr, no_curr)
# all_ratings


In [42]:
# z = get_similar_baseline('17 Again (2009)',4,0)
# z

In [43]:
# z['17 Again (2009)'] = 5
# z.index[0]

In [44]:
# avg_rating = sum_ratings/sum_responses

In [45]:
# def get_similar_baseline(movie_name,rating,row):
#     row = ratings.iloc[row, :].items()
#     user_ratings = [ rating for title, rating in row if rating != 0.0]
#     avg_user_rating = sum(user_ratings)/len(user_ratings)
#     similar_ratings = corrMatrix[movie_name]*(rating - 2.5 -(3 * avg_rating - avg_user_rating - all_ratings[movie_name][0]/all_ratings[movie_name][1]))
#     # for cols in similar_ratings.index:
#     #     similar_ratings[cols] += 3 * avg_rating - avg_user_rating - all_ratings[cols][0]/all_ratings[cols][1]
#     similar_ratings = similar_ratings.sort_values(ascending=False)
#     # print("Similar Ratings", similar_ratings)
#     return similar_ratings

In [46]:
# k = 10
# precisions, recalls, f1_scores = 0, 0, 0
# for rows in range(1):
#         print(rows)
#         # to get actual movies can be contain in actual top-k-rated movies
#         actual_ratings = [(title, rating) for title, rating in ratings.iloc[rows, :].items() if rating != 0.0]
#         actual_ratings_sorted = sorted(actual_ratings, key=lambda x: x[1], reverse=True)
#         top_k_min_rating = min([rating for _, rating in actual_ratings_sorted[:k]])
#         Actual_top_rated_movies = [movie for movie, rating in actual_ratings_sorted if rating >= top_k_min_rating]
    
#         # to predict to k movies
#         similar_movies = pd.DataFrame(columns = ratings.columns)
#         for movie,rating in actual_ratings_sorted:
#             similar_movies.loc[len(similar_movies)] = get_similar_baseline(movie,rating,rows)
        
#         top_predicted_movies_Series = similar_movies.sum().sort_values(ascending=False).head(10)
#         top_predicted_movies = top_predicted_movies_Series.index.tolist()
    
#         true_positives = len(set(Actual_top_rated_movies) & set(top_predicted_movies))
#         false_positives = len(set(top_predicted_movies) - set(Actual_top_rated_movies))
#         false_negatives = len(set(Actual_top_rated_movies) - set(top_predicted_movies))
    
#         precision = 0 if true_positives == 0 else true_positives / (true_positives + false_positives)
#         recall = 0 if true_positives == 0 else true_positives / (true_positives + false_negatives)
#         f1_score = 0 if true_positives == 0 else 2 * precision * recall / (precision + recall)
        
#         precisions += precision
#         recalls += recall
#         f1_scores += f1_score
    
# print ("K =", k, "Avg. Precision", precisions/len(ratings), "Avg. Recall", recalls/len(ratings), "Avg_f1", f1_scores/len(ratings))

In [47]:
# get_similar_baseline('Adventures of Tintin, The (2011)',3,1)

In [48]:
# top_predicted_movies_Series 